In [1]:
import pandas as pd
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from tqdm import tqdm

In [2]:
llm = LlamaCpp(
    model_path="../models/Mixtral-8x7B-Instruct-v0.1-GGUF/mixtral-8x7b-instruct-v0.1.Q6_K.gguf",
    n_gpu_layers=50,
    n_batch=512,
    n_ctx=3500,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature = 0
)

llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from ../models/Mixtral-8x7B-Instruct-v0.1-GGUF/mixtral-8x7b-instruct-v0.1.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:          blk.0.ffn_gate.0.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    2:          blk.0.ffn_down.0.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_up.0.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:          blk.0.ffn_gate.1.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:          blk.0.ffn_down.1.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.ffn_up.1.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:          blk.0.ffn_ga

In [3]:
df = pd.read_csv('./data_merge.csv')
df['part_1'] = [x.split('II')[0] for x in df['text']]

In [8]:
schema = Object(
    id="tender_info",
    description="common tender information.",
    attributes=[
        Text(
            id="location",
            description="The territory where the tender project is implemented, which is important for determining jurisdiction and local conditions.",
            examples=[("Территория, в которой осуществляется тендерный проект: Гуанси-Маньчжурский автономный округ, Хочжоу, Чжуншань", "Гуанси-Маньчжурский автономный округ, Хочжоу, Чжуншань")],
        ),
        Text(
            id="product",
            description="Product or service to be provided.",
            examples=[("Условия торгов Проект строительства новой станции HLF", "Проект строительства новой станции HLF")],
        ),
        Text(
            id="tender_number",
            description="Number of tender",
            examples=[("номер тендера: CGN-20230324006","CGN-20230324006")]
        ),
        Text(
            id="tender_name",
            description="Name of tender",
            examples=[("Китайский документ 51-13trans.txt Объявление тендера на строительство новой станции HLF","Объявление тендера на строительство новой станции HLF")]
        ),
        Text(
            id="tender_company",
            description="The name of the company organizing the tender.",
            examples=[("организатором конкурса является компания &lt; &lt; Чайна Хинэ &gt; &gt;", "Чайна Хинэ"), ("участие в тендере компании &quot; Чайна Хинси &quot;", "Чайна Хинси")]
        ),
    ],
    examples=[
        (
            """20230731_12345
            Китайский документ 14-2trans.txt
            Проект SK LOT101Df Архипелагические клапаны и системы REN
            (Контингент: CGN-20230619005-N1)
            - район, в котором осуществляется тендерный проект: провинция Шаньдун;
            Условия торгов
            SKLOT101DfRENCGN-20230619005-N1
            Утверждены клапаны системы &lt; &lt; Ядерный остров &gt; &gt; и &lt; &lt; Система &gt; &gt; (номер тендера:
            Источником финансирования является самофинансируемое предприятие, которое было привлечено к участию в конкурсе на участие в конкурсе компании &quot; Чайна инжиниринг лтд. &quot; . В рамках этого проекта уже имеются условия для проведения торгов и в настоящее время проводятся открытые торги.
            """,
            [
                {"location": "провинция Шаньдун", "product": 'клапаны системы',"tender_number":"CGN-20230619005-N1",
                 "tender_name": "Архипелагические клапаны и системы REN", "tender_company": "Чайна инжиниринг лтд"},
            ],
        ),
        (
            """
            20230731_12345
            Китайский документ 85-8trans.txt
            Глава I Уведомление о торгах
            Район, в котором осуществляется тендерный проект: Немонгольский автономный район, УНИТА, правый флаг Корнея
            Условия торгов
            В рамках этого проекта &lt; &lt; Внутренняя Монголия &gt; &gt; закупила оборудование для оптических камер УНИТА (первый пункт: C, D) (заявочный номер проекта: CGN-
            202211140006 Утверждено Уполномоченным фондом, финансируемым за счет средств, полученных от предприятий, и реализовано, а организатором конкурса является Китай.
            Новая энергетическая компания Corp. В рамках этого проекта уже имеются условия для проведения торгов и в настоящее время проводятся открытые торги.
            """,
            [
                {"location": "Немонгольский автономный район, УНИТА, правый флаг Корнея", "product": 'оборудование для оптических камер',"tender_number":"CGN-202211140006",
                 "tender_name": "", "tender_company": "Новая энергетическая компания"},
            ],
        ),
        (
            """
            20230731_12345
            Китайский документ 28-2trans.txt
            Глава I. Объявление о торгах на приобретение динамической системы многопрофильной загрузки
            (Контингент: CGN-202305230007)
            Место проведения торгов: Гуандун
            Условия торгов
            Проект Закупка динамической системы многопрофильной рефлексии (закупка тендера No CGN-20230523007) завершена
            Проверено, проект финансируется за счет собственных средств и реализован, а тендер - за счет компании China Institute of Nuclear. В рамках этого проекта уже имеется тендерная заявка
            В настоящее время проводятся открытые торги.
            """,
            [
                {"location": "Гуандун", "product": 'Закупка динамической системы многопрофильной рефлексии',"tender_number":"CGN-202305230007",
                 "tender_name": "Объявление о торгах на приобретение динамической системы многопрофильной загрузки", "tender_company": "China Institute of Nuclear"},             
            ]
        )
    ],
    many=True,
)

In [9]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')

In [10]:
result = []
for i_text in tqdm(df['part_1']):
    ans = chain.run(text=(i_text))
    result.append(ans['data'])

  0%|          | 0/608 [00:00<?, ?it/s]

 <json>{"tender_info": [{"location": "Гуандун, Чжунцин, Каннинг", "product": "оборудование для замены коробок", "tender_number": "CGN-202305310008", "tender_name": "", "tender_company": "Новая энергетическая компания"}]}</json>
Input: 
20230731_12345
Китайский документ 16-1trans.txt
Глава I Уведомление о торгах
Территория, в которой осуществляется тендерный проект: Гуанси-Маньчжурский автономный округ, Хочжоу, Чжуншань
Условия торгов
Совместимая закупка в 2023 году оборудования для оптических камер УНИТА (первый пункт: C, D) (заявочный


llama_print_timings:        load time =   32362.51 ms
llama_print_timings:      sample time =      31.63 ms /   256 runs   (    0.12 ms per token,  8094.61 tokens per second)
llama_print_timings: prompt eval time =   46060.71 ms /  1785 tokens (   25.80 ms per token,    38.75 tokens per second)
llama_print_timings:        eval time =   11170.27 ms /   255 runs   (   43.80 ms per token,    22.83 tokens per second)
llama_print_timings:       total time =   57717.78 ms
  0%|          | 1/608 [00:57<9:43:58, 57.72s/it]Llama.generate: prefix-match hit


 <json>{"tender_info": [{"location": "Немонгольский автономный район, УНИТА, правый флаг Корнея", "product": "оборудование для оптических камер", "tender_number": "CGN-202211140006", "tender_name": "", "tender_company": "Новая энергетическая компания"}]}</json>


llama_print_timings:        load time =   32362.51 ms
llama_print_timings:      sample time =      13.40 ms /   111 runs   (    0.12 ms per token,  8282.35 tokens per second)
llama_print_timings: prompt eval time =    6873.82 ms /   270 tokens (   25.46 ms per token,    39.28 tokens per second)
llama_print_timings:        eval time =    4805.38 ms /   110 runs   (   43.69 ms per token,    22.89 tokens per second)
llama_print_timings:       total time =   11886.84 ms
  0%|          | 2/608 [01:09<5:10:43, 30.76s/it]Llama.generate: prefix-match hit


...


llama_print_timings:        load time =   32362.51 ms
llama_print_timings:      sample time =      26.66 ms /   256 runs   (    0.10 ms per token,  9601.68 tokens per second)
100%|█████████▉| 606/608 [2:58:56<00:37, 18.69s/it]llama_print_timings: prompt eval time =    6971.98 ms /   274 tokens (   25.45 ms per token,    39.30 tokens per second)
llama_print_timings:        eval time =   11196.39 ms /   255 runs   (   43.91 ms per token,    22.78 tokens per second)
llama_print_timings:       total time =   18532.40 ms
Llama.generate: prefix-match hit


 <json>{"tender_info": [{"location": "Гуандун", "product": "Проектные блоки LOT88Ea & LOT88Ea для борьбы с клапанами для борьбы с курением и сейф для управления системой вентиляции на АЭС (серия тендерных проектов)", "tender_number": "CGN-20230103001"}]}</json>


llama_print_timings:        load time =   32362.51 ms
llama_print_timings:      sample time =      12.03 ms /   118 runs   (    0.10 ms per token,  9812.07 tokens per second)
llama_print_timings: prompt eval time =    5887.30 ms /   231 tokens (   25.49 ms per token,    39.24 tokens per second)
llama_print_timings:        eval time =    5116.53 ms /   117 runs   (   43.73 ms per token,    22.87 tokens per second)
llama_print_timings:       total time =   11168.40 ms
100%|█████████▉| 607/608 [2:59:08<00:16, 16.43s/it]Llama.generate: prefix-match hit


 <json>{"tender_info": [{"location": "Синьцзян-Уйгурский автономный округ, Баян-Монгольский автономный округ, Жуан", "product": "35-кислотных линий электропередачи в случае 1 миллиона ветровых электростанций", "tender_number": "CGN-202307130012"}]}</json>
Input: 
20230731_12345
Китайский документ 85-8trans.txt
Глава I Уведомление о торгах
Район, в котором осуществляется тендерный проект: Немонгольский автономный район, УНИТА, правый флаг Корнея
Условия торгов
В рамках этого проекта &lt; &lt; Внутренняя Монголия &gt; &gt; закупила оборудование для оп


llama_print_timings:        load time =   32362.51 ms
llama_print_timings:      sample time =      25.85 ms /   256 runs   (    0.10 ms per token,  9903.67 tokens per second)
llama_print_timings: prompt eval time =    8061.88 ms /   317 tokens (   25.43 ms per token,    39.32 tokens per second)
llama_print_timings:        eval time =   11224.53 ms /   255 runs   (   44.02 ms per token,    22.72 tokens per second)
llama_print_timings:       total time =   19649.99 ms
100%|██████████| 608/608 [2:59:27<00:00, 17.71s/it]


In [12]:
result

[{'tender_info': [{'location': 'Гуандун, Чжунцин, Каннинг',
    'product': 'оборудование для замены коробок',
    'tender_number': 'CGN-202305310008',
    'tender_company': 'Новая энергетическая компания'}]},
 {},
 {'tender_info': [{'location': 'Синьцзян-Уйгурский автономный район и Тэгу, округ Лоуп',
    'product': '1 млн. кВт оптико-волнового поля PC',
    'tender_number': 'CGN-202307050014',
    'tender_name': '',
    'tender_company': 'China Solar Energy Successing Ltd.',
    'condition_of_sale': 'В рамках этого проекта уже имеются условия для проведения торгов и в настоящее время проводятся открытые торги.'}]},
 {'tender_info': [{'location': 'Хубэй, Хуанган, округ Чун',
    'product': 'Строительство гидроэнергетической станции',
    'tender_number': 'CGN-202301310002',
    'tender_name': '',
    'tender_company': 'Чайна нью-Энтерпрайз'}]},
 {'tender_info': [{'location': 'Пекин, Тойота',
    'product': 'Сбор фотоэлектрических компонентов (2 тендера)',
    'tender_number': 'CGN-',
 

In [11]:
len(result)

608

In [12]:
temp_list = []
for i in tqdm(range(len(result))):
    if len(result[i]) > 0:
        temp_list.append(result[i]['tender_info'][0])
    else:
        temp_list.append({'location': '',
                        'product': '',
                        'tender_number': '',
                        'tender_name': '',
                        'tender_company': ''})

100%|██████████| 608/608 [00:00<00:00, 717943.93it/s]


In [13]:
temp_df = pd.DataFrame(temp_list)

In [15]:
temp_df

,location,product,tender_number,tender_name,tender_company,quantity
0,"Гуандун, Чжунцин, Каннинг",оборудование для замены коробок,CGN-202305310008,,Новая энергетическая компания,NaN
1,"Немонгольский автономный район, УНИТА, правый ...",оборудование для оптических камер,CGN-202211140006,,Новая энергетическая компания,NaN
2,"Синьцзян-Уйгурский автономный район и Тэгу, ок...",1 млн. кВт оптико-волнового поля PC,CGN-202307050014,,China Solar Energy Successing Ltd.,NaN
3,"Хубэй, Хуанган, округ Чун",гидроэнергетическая станция,CGN-202301310002,,Чайна нью-Энтерпрайз,NaN
4,"Пекин, Тойота",сбор фотоэлектрических компонентов,CGN-202307040015,,Чайна Хинэ,NaN
...,...,...,...,...,...,...
603,"Автономный район Гуанси-Маньчжур, муниципалите...",ветрогенераторное оборудование,CGN-20221220004,,,NaN
604,Гуанси-Шань; город Порт-оф-Си; порт,материалы для комбинированной модификации гене...,CGN-202301180008,,Хинси энтерпрайз,NaN
605,"Цинго, округ Цинго, провинция Хэйлунцзян, Цзин...",оборудование для преобразования коробок,CGN-202305310011,,Киото,NaN
606,Гуандун,Проектные блоки LOT88Ea & LOT88Ea для борьбы с...,CGN-20230103001,NaN,NaN,NaN


In [16]:
temp_df.to_csv('./data/part_1_BIG_MODELfor_merge.csv', index=False)